# 5. 토큰화 Tokenizing

## 5-a. 토큰화 계획
- word extraction 한 단어 리스트로 사용자 정의 사전을 만든다
- mecab 이용하여 tokenizing 한다

## 5-b. 토큰화 프로세스
1. mecab 을 컴파일하고, 테스트한다
  - 테스트할 text 를 제작한다
  - 토큰화 테스트
2. word extraction의 결과 단어 리스트를 불러온다
3. 사용자 정의 사전을 제작하고, .csv 파일로 저장해둔다
  - 종성이 있는지 판단한다
4. 여기까지 함수화한다 (단어 리스트 불러오기 + 사용자 정의 사전 파일로 저장)
5. 사용자 정의 사전을 `C:\mecab\user-dic` 에 옮긴다
6. 사용자 정의 사전을 컴파일한다
7. 토큰화 한다
  - [1-a] 의 text로 테스트를 진행한다
  - 변화가 있는지 체크한다

## 5-c. 사용자 정의사전 만들기

### 4-c-1. mecab 컴파일
- Mecab 메뉴얼 : https://kain7f1.notion.site/Mecab-195acbade3f14463b137c4490baebc76?pvs=4

### 5-c-2. (pycharm으로 실행) macab 토큰화 테스트

In [4]:
# 메뉴얼에 따라 빌드하고, pycharm에서 테스트합니다
import MeCab

txt1 = "주식으로 선거때마다 한탕 해처먹는 간잽이진짜 확망했으면"
m = MeCab.Tagger()
tokens = m.parse(txt1)
print(tokens)

ModuleNotFoundError: No module named 'MeCab'

### 5-c-3. word extraction의 결과 단어 리스트를 불러오기

In [1]:
# 텍스트 파일을 열고 읽기
file_path = "4_ecopro_word_list.txt"
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# 줄바꿈 문자 제거
word_list = [line.strip() for line in lines]

# 결과 출력
print(word_list)

['현재', '없어', '예정', '결정', '바로', '않음', '이거', '입갤', '잔고', '버핏', '중에', 'ㅅㄱ', '반등', '지수', '하면', '모든', '수익', '서울', '살까', '당장', '해도', '운지', '형님', '조정', '할거', '흐름', '진입', '층에', '흑우', '경쟁', '제가', '증가', '갈때', '제일', '졸업', '인데', '높음', '맨날', '매수', '장기', '털고', '잡고', '거의', '병신', '좋아', '구조', '기관', '길게', '피자', '성장', '결국', '박살', '뉴스', '별로', '샀어', '특징', '대한', '않나', '맞냐', '소재', '난리', '찍고', '본다', '몇개', '랠리', '양극', '믿는', '심리', '갔다', '똑같', '방어', '회복', '규모', '밸류', '빠진', '맞음', '매매', '바닥', '되냐', '수준', '올해', '이런', '방법', '너네', '축하', '누구', '무슨', '관심', '해서', '벌써', '섹터', '엘앤', '큐ㅠ', '급락', '언제', '했다', '에코', '테마', '추매', '정리', '억원', '부자', '쉽게', '그냥', '달러', '방금', '없다', '하락', '봐야', '승리', '되나', '맞다', 'ㅆㅂ', '너무', '확률', '직원', '소리', '싸게', '벌고', '연상', '많음', '재미', '허허', '물린', '분명', '은행', 'ㅅㅂ', '머리', '캬ㅋ', '되지', '슈카', '다음', '걱정', '적정', '형들', '앞으', '구간', '살껄', '상승', '넘게', '삼성', '포홀', '국내', '영익', '그런', '산업', '보고', '미래', '남들', '굳이', '오를', '분기', '크게', 'ㅎㅎ', '팔면', '진짜', '확정', '두산', '선동', '유증', '전부', '백만', '버스', '저점', '호가', '타점', '액분

In [2]:
len(word_list)

1061

### 5-c-4. 사용자 정의 사전 만들기

In [3]:
import numpy as np
import pandas as pd

# 단어의 마지막 글자에 종성이 있는지 체크하는 함수
def has_jongseong(char):
    if '가' <= char <= '힣':
        unicode_value = ord(char)
        jongseong = (unicode_value - 0xAC00) % 28
        return jongseong != 0
    else:
        return False

data = []  # 사전에 들어갈 형태로 가공해서 넣을거임
for word in word_list:
    if len(word) == 0:
        continue
    if has_jongseong(word[-1]):  # 마지막 글자에 종성이 있으면 'T' 없으면 'F'
        jongseong = 'T'
    else:
        jongseong = 'F'
    # 사전 파일인 nnp.csv에 맞는 형식으로 데이터를 세팅함
    new_row = [word, np.nan, np.nan, 0, 'NNP', '*', jongseong, word, '*', '*', '*', '*']
    data.append(new_row)

df = pd.DataFrame(data)

user_dict_annlab.csv 파일이 생성되었습니다


In [4]:
fpath_to_save = 'user_dict_ecopro.csv'
df.to_csv(fpath_to_save, index=False, encoding="utf-8", header=False)
print(f'{fpath_to_save} 파일이 생성되었습니다')

user_dict_ecopro.csv 파일이 생성되었습니다


### 5-c-5. 사용자 정의 사전 제작 함수화

In [1]:
import numpy as np
import pandas as pd

def make_user_dict(fpath_word_list, fpath_user_dict):
    # 1. word_list 파일 읽어오기
    print("[progress 1/3] word_list 파일 읽어오기")
    with open(fpath_word_list, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    word_list = [line.strip() for line in lines]  # 줄바꿈 문자 제거

    # 단어의 마지막 글자에 종성이 있는지 체크하는 함수
    def has_jongseong(char):
        if '가' <= char <= '힣':
            unicode_value = ord(char)
            jongseong = (unicode_value - 0xAC00) % 28
            return jongseong != 0
        else:
            return False

    # 2. mecab 사전의 형식으로 데이터 가공하기
    print("[progress 2/3] mecab 사전의 형식으로 데이터 가공하기")
    data = []  # 사전에 들어갈 형태로 가공해서 넣을거임
    for word in word_list:
        if len(word) == 0:
            continue
        if has_jongseong(word[-1]):  # 마지막 글자에 종성이 있으면 'T' 없으면 'F'
            jongseong = 'T'
        else:
            jongseong = 'F'
        # 사전 파일인 nnp.csv에 맞는 형식으로 데이터를 세팅함
        new_row = [word, np.nan, np.nan, 0, 'NNP', '*', jongseong, word, '*', '*', '*', '*']
        data.append(new_row)
    
    df = pd.DataFrame(data)

    # 3. 가공한 데이터를 파일로 저장하기
    print("[progress 3/3] 가공한 데이터를 파일로 저장하기")
    df.to_csv(fpath_user_dict, index=False, encoding="utf-8", header=False)
    print(f"{fpath_user_dict} 파일이 생성되었습니다")

    print("[done]")

### 5-c-6. 함수 테스트

In [2]:
fpath_word_list = "4_ecopro_word_list.txt"
fpath_user_dict = "user_dict_ecopro.csv"

make_user_dict(fpath_word_list, fpath_user_dict)

[progress 1/3] word_list 파일 읽어오기
[progress 2/3] mecab 사전의 형식으로 데이터 가공하기
[progress 3/3] 가공한 데이터를 파일로 저장하기
user_dict_ecopro.csv 파일이 생성되었습니다
[done]


## 5-d. 사용자 정의 사전 컴파일
- 사용자 정의 사전을 옮긴다
- Windows ProwerShell ISE 를 켠다
- cd C:\mecab
- .\tools\add-userdic-win.ps1

## 5-e. 토큰화 Tokenizing

### 5-e-1. 토큰화는 test_tokenizing.py 로 진행한다
### 5-e-2. tokens 칼럼이 생성된 파일을 저장한다